In [39]:
import xarray as xr
import pandas as pd
import urllib.request
import numpy as np

In [40]:
numfiles = 85
url = 'https://www.ncei.noaa.gov/data/north-american-mesoscale-model/access/forecast/202207/20220723/'
file_prefix = 'nam_218_20220723_0000_0'
file_post = '.grb2'


In [ ]:

#download all of the grib files... this could take a while
for i in range(numfiles):
    urllib.request.urlretrieve(url+file_prefix+str(i).zfill(2)+file_post, filename=file_prefix+str(i).zfill(2)+file_post)
    

In [41]:
#I lifted this code from... https://github.com/blaylockbk/Ute_WRF/blob/master/functions/wind_calcs.py

def wind_uv_to_dir(U,V):
    """
    Calculates the wind direction from the u and v component of wind.
    Takes into account the wind direction coordinates is different than the 
    trig unit circle coordinate. If the wind directin is 360 then returns zero
    (by %360)
    Inputs:
      U = west/east direction (wind from the west is positive, from the east is negative)
      V = south/noth direction (wind from the south is positive, from the north is negative)
    """
    WDIR= (270-np.rad2deg(np.arctan2(V,U)))%360
    return WDIR
    
def wind_uv_to_spd(U,V):
    """
    Calculates the wind speed from the u and v wind components
    Inputs:
      U = west/east direction (wind from the west is positive, from the east is negative)
      V = south/noth direction (wind from the south is positive, from the north is negative)
    """
    #convert to mph from mps with constant 2.23694
    WSPD = np.sqrt(np.square(U)+np.square(V)) * 2.23694
    return WSPD

In [115]:
import os
file_prefix = 'nam_218_20220723_0000_0'
file_post = '.grb2'

os.chdir('/home/eturner/Downloads')
ds = xr.open_dataset('nam_218_20220319_0000_003.grb2'+file_post,engine='pynio')

In [116]:
ds =ds.get(["UGRD_P0_L103_GLC0","VGRD_P0_L103_GLC0"])


In [117]:
df = ds.to_dataframe()

In [118]:
#use this code to find all the codes inside the file.  For a full list of these codes with better 
#descriptions look here:
#https://rapidrefresh.noaa.gov/GRIB2Table_GSDrap_2d.txt
#specifically, we want the winds at 10m above ground -which is the U and V UGRD_P0_L103_GLC0 codes.
    
for v in ds:
    print("{}, {}, {}".format(v, ds[v].attrs["long_name"], ds[v].attrs["units"]))

UGRD_P0_L103_GLC0, U-component of wind, m s-1
VGRD_P0_L103_GLC0, V-component of wind, m s-1


In [119]:
df.info()


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 525584 entries, (10.0, 0, 0) to (80.0, 427, 613)
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   UGRD_P0_L103_GLC0  525584 non-null  float32
 1   VGRD_P0_L103_GLC0  525584 non-null  float32
 2   gridlat_0          525584 non-null  float32
 3   gridlon_0          525584 non-null  float32
dtypes: float32(4)
memory usage: 10.5 MB


In [120]:
df2 = df[df['gridlat_0'] < 28.454]
df3 = df2[df2['gridlat_0'] > 28.452]

In [121]:
df3

UGRD_P0_L103_GLC0  VGRD_P0_L103_GLC0  gridlat_0  \
lv_HTGL3 ygrid_0 xgrid_0                                                    
10.0     100     327              -1.380873          -1.653447  28.453312   
                 366               5.539127          -5.243447  28.453930   
         101     296               0.759127          -2.703447  28.453176   
         104     442              -0.570873           2.926553  28.452951   
         111     191               2.829127          -3.463447  28.453079   
         117     154               1.259127          -7.423447  28.453733   
         120     138              -0.260873          -7.103447  28.453850   
         122     128              -0.600873          -6.503447  28.453804   
         126     584              -0.160873          -0.323447  28.453833   
         130     92               -1.500873          -3.653447  28.452278   
         141     50               -2.170873          -3.763447  28.453436   
         143     43               -1.710873          -3.523447  28.452734   
80.0     100     327              -4.475610          -5.070876  28.453312   
                 366               6.194389          -6.000876  28.453930   
         101     296              -3.165610          -6.530876  28.453176   
         104     442               2.164390           7.049123  28.452951   
         111     191               3.554389          -4.620876  28.453079   
         117     154               1.384390          -8.570876  28.453733   
         120     138              -0.355610          -8.100876  28.453850   
         122     128              -0.715610          -7.190876  28.453804   
         126     584              -0.015610          -0.300876  28.453833   
         130     92               -1.625610          -3.980876  28.452278   
         141     50               -2.345610          -4.100876  28.453436   
         143     43               -1.885610          -3.860876  28.452734   

                           gridlon_0  
lv_HTGL3 ygrid_0 xgrid_0              
10.0     100     327      -97.446884  
                 366      -92.592438  
         101     296     -101.307472  
         104     442      -83.117973  
         111     191     -114.440674  
         117     154     -119.106316  
         120     138     -121.132507  
         122     128     -122.401825  
         126     584      -65.219910  
         130     92      -126.992119  
         141     50      -132.396347  
         143     43      -133.302475  
80.0     100     327      -97.446884  
                 366      -92.592438  
         101     296     -101.307472  
         104     442      -83.117973  
         111     191     -114.440674  
         117     154     -119.106316  
         120     138     -121.132507  
         122     128     -122.401825  
         126     584      -65.219910  
         130     92      -126.992119  
         141     50      -132.396347  
         143     43      -133.302475

In [122]:
twdb_stations = pd.read_csv('/home/eturner/coastal-spill/Winds/NAMwinds.latlist.csv')

In [123]:
twdb_stations.head()

,station,lat,lon
0,twdb000,28.453,-97.448
1,twdb001,28.455,-97.324
2,twdb002,28.339,-97.695
3,twdb003,28.341,-97.570
4,twdb004,28.344,-97.446


In [124]:
winds = pd.DataFrame()
arrays = []
for index, row in twdb_stations.iterrows():
    print ("searching for... ",row['station'],row["lat"], row['lon'])
    #print(row["station"], row["lat"], row['lon'])
    df2 = df[df['gridlat_0'] < (row['lat']+.001)]
    df3 = df2[df2['gridlat_0'] > (row['lat'] - .001)]
    df4 = df3[df3['gridlon_0'] < (row['lon'] + .01)]
    df5 = df4[df4['gridlon_0'] > (row['lon'] - .01)]
    row_1=df5.iloc[0]

    print ("found ", len(df5), " matches with row 1 as:", df5['gridlat_0'].iat[0], df5['gridlon_0'].iat[0])
    print ("row two as ",df5['gridlat_0'].iat[1], df5['gridlon_0'].iat[1])
    located = df5.iloc[:1]
    located['station'] = row['station']
    located['org_lat'] = row['lat']
    located['org_lon'] = row['lon']
    located['speed'] = wind_uv_to_spd(located['UGRD_P0_L103_GLC0'].iat[0],located['VGRD_P0_L103_GLC0'].iat[0]) 
    located['dir'] = wind_uv_to_dir(located['UGRD_P0_L103_GLC0'].iat[0],located['VGRD_P0_L103_GLC0'].iat[0])
    arrays.append(located)
    #winds.append(df5)
    

searching for...  twdb000 28.453 -97.448
found  2  matches with row 1 as: 28.453312 -97.446884
row two as  28.453312 -97.446884
searching for...  twdb001 28.455 -97.324
found  2  matches with row 1 as: 28.455236 -97.32244
row two as  28.455236 -97.32244
searching for...  twdb002 28.339 -97.695
found  2  matches with row 1 as: 28.339745 -97.6933
row two as  28.339745 -97.6933
searching for...  twdb003 28.341 -97.57
found  2  matches with row 1 as: 28.341867 -97.56897
row two as  28.341867 -97.56897
searching for...  twdb004 28.344 -97.446
found  2  matches with row 1 as: 28.343891 -97.44464
row two as  28.343891 -97.44464
searching for...  twdb005 28.345 -97.322
found  2  matches with row 1 as: 28.345814 -97.32031
row two as  28.345814 -97.32031
searching for...  twdb006 28.347 -97.197
found  2  matches with row 1 as: 28.347637 -97.19597
row two as  28.347637 -97.19597
searching for...  twdb007 28.228 -97.816
found  2  matches with row 1 as: 28.228092 -97.81504
row two as  28.228092 -97

/tmp/ipykernel_148572/3430608072.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['station'] = row['station']
/tmp/ipykernel_148572/3430608072.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['org_lat'] = row['lat']
/tmp/ipykernel_148572/3430608072.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

 2  matches with row 1 as: 27.901958 -97.68346
row two as  27.901958 -97.68346
searching for...  twdb039 27.904 -97.561
found  2  matches with row 1 as: 27.904076 -97.559586
row two as  27.904076 -97.559586
searching for...  twdb040 27.906 -97.437
found  2  matches with row 1 as: 27.906092 -97.43571
row two as  27.906092 -97.43571
searching for...  twdb041 27.908 -97.313
found  2  matches with row 1 as: 27.908009 -97.31183
row two as  27.908009 -97.31183
searching for...  twdb042 27.909 -97.189
found  2  matches with row 1 as: 27.909826 -97.18794
row two as  27.909826 -97.18794
searching for...  twdb043 27.911 -97.065
found  2  matches with row 1 as: 27.911543 -97.06406
row two as  27.911543 -97.06406
searching for...  twdb044 27.913 -96.941
found  2  matches with row 1 as: 27.91316 -96.94017
row two as  27.91316 -96.94017
searching for...  twdb045 27.785 -98.053
found  2  matches with row 1 as: 27.785538 -98.05225
row two as  27.785538 -98.05225
searching for...  twdb046 27.788 -97.93

/tmp/ipykernel_148572/3430608072.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['station'] = row['station']
/tmp/ipykernel_148572/3430608072.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['org_lat'] = row['lat']
/tmp/ipykernel_148572/3430608072.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

found  2  matches with row 1 as: 27.583065 -97.05841
row two as  27.583065 -97.05841
searching for...  twdb075 27.584 -96.936
found  2  matches with row 1 as: 27.584679 -96.93486
row two as  27.584679 -96.93486
searching for...  twdb076 27.586 -96.812
found  2  matches with row 1 as: 27.586191 -96.81131
row two as  27.586191 -96.81131
searching for...  twdb077 27.587 -96.689
found  2  matches with row 1 as: 27.587605 -96.68775
row two as  27.587605 -96.68775
searching for...  twdb078 27.461 -97.798
found  2  matches with row 1 as: 27.46179 -97.7971
row two as  27.46179 -97.7971
searching for...  twdb079 27.464 -97.675
found  2  matches with row 1 as: 27.463999 -97.67368
row two as  27.463999 -97.67368
searching for...  twdb080 27.466 -97.551
found  2  matches with row 1 as: 27.466108 -97.55026
row two as  27.466108 -97.55026
searching for...  twdb081 27.468 -97.428
found  2  matches with row 1 as: 27.46812 -97.426834
row two as  27.46812 -97.426834
searching for...  twdb082 27.47 -97.3

...

/tmp/ipykernel_148572/3430608072.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['org_lat'] = row['lat']
/tmp/ipykernel_148572/3430608072.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['org_lon'] = row['lon']
/tmp/ipykernel_148572/3430608072.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

searching for...  twdb257 29.128 -94.583
found  2  matches with row 1 as: 29.128489 -94.581665
row two as  29.128489 -94.581665
searching for...  twdb258 29.128 -94.458
found  2  matches with row 1 as: 29.128101 -94.45649
row two as  29.128101 -94.45649
searching for...  twdb259 29.127 -94.333
found  2  matches with row 1 as: 29.127611 -94.331314
row two as  29.127611 -94.331314
searching for...  twdb260 29.127 -94.207
found  2  matches with row 1 as: 29.127022 -94.20614
row two as  29.127022 -94.20614
searching for...  twdb261 29.126 -94.082
found  2  matches with row 1 as: 29.126331 -94.08097
row two as  29.126331 -94.08097
searching for...  twdb262 29.016 -95.959
found  2  matches with row 1 as: 29.016743 -95.957695
row two as  29.016743 -95.957695
searching for...  twdb263 29.017 -95.834
found  2  matches with row 1 as: 29.017464 -95.83264
row two as  29.017464 -95.83264
searching for...  twdb264 29.018 -95.709
found  2  matches with row 1 as: 29.018085 -95.70759
row two as  29.018

/tmp/ipykernel_148572/3430608072.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['station'] = row['station']
/tmp/ipykernel_148572/3430608072.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['org_lat'] = row['lat']
/tmp/ipykernel_148572/3430608072.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

found  2  matches with row 1 as: 28.690464 -95.58092
row two as  28.690464 -95.58092
searching for...  twdb302 28.691 -95.457
found  2  matches with row 1 as: 28.690884 -95.45621
row two as  28.690884 -95.45621
searching for...  twdb303 28.691 -95.333
found  2  matches with row 1 as: 28.6912 -95.33149
row two as  28.6912 -95.33149
searching for...  twdb304 28.691 -95.208
found  2  matches with row 1 as: 28.691418 -95.20678
row two as  28.691418 -95.20678
searching for...  twdb305 28.691 -95.083
found  2  matches with row 1 as: 28.691534 -95.08206
row two as  28.691534 -95.08206
searching for...  twdb306 28.691 -94.959
found  2  matches with row 1 as: 28.691551 -94.957344
row two as  28.691551 -94.957344
searching for...  twdb307 28.691 -94.834
found  2  matches with row 1 as: 28.691465 -94.832634
row two as  28.691465 -94.832634
searching for...  twdb308 28.691 -94.709
found  2  matches with row 1 as: 28.69128 -94.70792
row two as  28.69128 -94.70792
searching for...  twdb309 28.581 -9

/tmp/ipykernel_148572/3430608072.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['station'] = row['station']
/tmp/ipykernel_148572/3430608072.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  located['org_lat'] = row['lat']
/tmp/ipykernel_148572/3430608072.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [125]:
winds = pd.DataFrame()


In [126]:
winds = pd.concat(arrays)

In [127]:
#twdb0001  tamu file  (spd mph, deg)
#2022 03 19 00    8.1       3.5
#2022 03 19 00    8.055183	5.408508  - from 'nam_218_20220319_0000_000.grb2
#2022 03 19 03    4.8       33.9
#2022 03 19 00    4.781781	35.814590   - from 'nam_218_20220319_0000_003.grb2
#2022 03 20 06    0.9       173.3
#2022 03 20 06    0.903721	175.533951 - from 'nam_218_20220320_0600_000.grb2
#2022 03 20 09    3.9       5.2
#2022 03 20 09    3.918048	7.138939  - from 'nam_218_20220320_0600_003.grb2


#twdb337 tamu file (spd mph, deg)
#2022 03 19 00    11.6      322.5
#2022 03 19 00    11.555776	321.715862 - from 'nam_218_20220319_0000_000.grb2
#2022 03 19 03    7.7       333.7
#2022 03 19 03    7.669285	332.950584   - from 'nam_218_20220319_0000_003.grb2
#2022 03 20 06    0.8       276.7
#2022 03 20 06    0.806347	275.927850 - from 'nam_218_20220320_0600_000.grb2
#2022 03 20 09    3.6       23.3
#2022 03 20 09    3.574614	22.350525  - from 'nam_218_20220320_0600_003.grb2

#twdb339 tamu file (spd mph, deg)
#2022 03 19 00    11.9      322.0
#2022 03 19 00    11.873649	320.997772 - from 'nam_218_20220319_0000_000.grb2
#2022 03 19 03    8.4       336.9
#2022 03 19 03    8.437221	335.916649  - from 'nam_218_20220319_0000_003.grb2
#2022 03 20 06    1.1       91.7
#2022 03 20 06    1.099381	90.323196 - from 'nam_218_20220320_0600_000.grb2
#2022 03 20 09    3.8       41.7
#2022 03 20 09    3.799991	40.696579 - from 'nam_218_20220320_0600_003.grb2

winds

UGRD_P0_L103_GLC0  VGRD_P0_L103_GLC0  gridlat_0  \
lv_HTGL3 ygrid_0 xgrid_0                                                    
10.0     100     327              -1.380873          -1.653447  28.453312   
                 328              -1.250873          -1.733447  28.455236   
         99      325              -1.760873          -1.803447  28.339745   
                 326              -1.650873          -1.843447  28.341867   
                 327              -1.490873          -1.873447  28.343891   
...                                     ...                ...        ...   
         112     358               1.499127          -3.553447  29.778305   
         111     354               1.559127          -3.053447  29.672844   
                 355               1.519127          -3.273447  29.672049   
                 356               1.539127          -3.443447  29.671152   
         110     355               1.889127          -3.653447  29.562780   

                          gridlon_0  station  org_lat  org_lon     speed  \
lv_HTGL3 ygrid_0 xgrid_0                                                   
10.0     100     327     -97.446884  twdb000   28.453  -97.448  4.818878   
                 328     -97.322441  twdb001   28.455  -97.324  4.781781   
         99      325     -97.693298  twdb002   28.339  -97.695  5.638284   
                 326     -97.568970  twdb003   28.341  -97.570  5.535547   
                 327     -97.444641  twdb004   28.344  -97.446  5.355828   
...                             ...      ...      ...      ...       ...   
         112     358     -93.572395  twdb336   29.778  -93.574  8.627275   
         111     354     -94.076706  twdb337   29.672  -94.078  7.669285   
                 355     -93.950958  twdb338   29.672  -93.952  8.072597   
                 356     -93.825211  twdb339   29.671  -93.826  8.437221   
         110     355     -93.951927  twdb340   29.562  -93.953  9.200455   

                                 dir  
lv_HTGL3 ygrid_0 xgrid_0              
10.0     100     327       39.866867  
                 328       35.814590  
         99      325       44.315643  
                 326       41.845566  
                 327       38.512466  
...                              ...  
         112     358      337.126106  
         111     354      332.950584  
                 355      335.105125  
                 356      335.916649  
         110     355      332.657387  

[341 rows x 9 columns]

In [101]:
winds['speed'].min()

0.3407793948355317